In [16]:
import json
import pandas as pd


In [17]:
file_path_mel = 'data_sets/bom_mel.json'



# 读取JSON文件
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data



mel_json = read_json_file(file_path_mel)

df_mel = pd.DataFrame(mel_json)

In [18]:
def clean_data(df):
    columns_drop = ['weather', 'swell_height', 'sea_state', 'swell_period', 'press_tend',
                     'swell_dir_worded', 'wmo', 'history_product', 'cloud_base_m', 'cloud_oktas', 
                     'cloud', 'cloud_type_id', 'cloud_type', 'aifstime_utc', 'local_date_time', 'sort_order', 'vis_km', 'rain_trace']
    df_cleaned = df.drop(columns = columns_drop)
    df_cleaned['name'] = 'Melbourne CBD'

    # 将 datetime 格式转换为 'YYYY-MM-DD' 格式
    df_cleaned['local_date_time_full'] = pd.to_datetime(df_cleaned['local_date_time_full'], format='%Y%m%d%H%M%S')
    df_cleaned['local_date_time_full'] = df_cleaned['local_date_time_full'].dt.strftime('%Y-%m-%d-%H')

    #这个是为了确保没有半个小时的数据出现
    if df_cleaned.iloc[0]['local_date_time_full'] != df_cleaned.iloc[1]['local_date_time_full']:
        df_cleaned = df_cleaned.drop(0)

    return df_cleaned


In [28]:
test = pd.read_csv("data_sets/merged_data.csv")
test['vis_km']

KeyError: 'vis_km'

In [19]:
def update_bom_first_two(df, filename):
    first_two_rows = df.head(2)
    df = clean_data(first_two_rows)
    # 将前两行数据附加到同一文件中
    df.to_csv(filename, mode='a', header=False, index=False)

In [20]:
df_mel_v2 = clean_data(df_mel)
df_mel_v2

,name,local_date_time_full,lat,lon,apparent_t,delta_t,gust_kmh,gust_kt,air_temp,dewpt,press,press_qnh,press_msl,rain_trace,rel_hum,vis_km,wind_dir,wind_spd_kmh,wind_spd_kt
1,Melbourne CBD,2024-05-13-16,-37.8,145.0,17.3,2.7,6,3,16.7,11.8,1022.1,1022.1,1022.1,0.0,73,10,CALM,0,0
2,Melbourne CBD,2024-05-13-16,-37.8,145.0,16.4,2.8,11,6,16.7,11.6,1022.0,1022.0,1022.0,0.0,72,10,S,4,2
3,Melbourne CBD,2024-05-13-15,-37.8,145.0,16.3,3.0,9,5,17.0,11.5,1022.0,1022.0,1022.0,0.0,70,10,SSW,6,3
4,Melbourne CBD,2024-05-13-15,-37.8,145.0,16.0,3.3,15,8,17.3,11.1,1021.7,1021.7,1021.7,0.0,67,10,SSW,9,5
5,Melbourne CBD,2024-05-13-14,-37.8,145.0,17.8,3.9,9,5,18.5,11.3,1021.4,1021.4,1021.4,0.0,63,10,S,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Melbourne CBD,2024-05-10-19,-37.8,145.0,15.8,0.0,6,3,14.7,14.7,1028.4,1028.4,1028.4,0.6,100,5,S,2,1
151,Melbourne CBD,2024-05-10-18,-37.8,145.0,16.8,0.0,0,0,15.1,15.1,1028.4,1028.4,1028.4,0.6,100,6,CALM,0,0
152,Melbourne CBD,2024-05-10-18,-37.8,145.0,17.0,0.0,0,0,15.3,15.3,1028.3,1028.3,1028.3,0.6,100,6,CALM,0,0
153,Melbourne CBD,2024-05-10-17,-37.8,145.0,17.0,0.0,0,0,15.3,15.3,1028.4,1028.4,1028.4,0.6,100,7,CALM,0,0


In [21]:
df_mel_v2['vis_km']

1      10
2      10
3      10
4      10
5      10
       ..
150     5
151     6
152     6
153     7
154    10
Name: vis_km, Length: 154, dtype: object

In [22]:
mask = df_mel_v2['local_date_time_full'].shift(-1) == df_mel_v2['local_date_time_full']
mask 

1       True
2      False
3       True
4      False
5       True
       ...  
150    False
151     True
152    False
153     True
154    False
Name: local_date_time_full, Length: 154, dtype: bool

In [23]:
# 初始化一个空的 DataFrame
def combine_1h(df_mel_v2):
    mask = df_mel_v2['local_date_time_full'].shift(-1) == df_mel_v2['local_date_time_full']
    processed_df = pd.DataFrame(columns=df_mel_v2.columns)

    # 对于连续的行，将其他列的值进行合并或者求平均
    rows_to_append = []
    for idx, row in df_mel_v2[mask].iterrows():
        if idx + 1 < len(df_mel_v2):
            # 合并 'local_date_time_full' 列
            row['local_date_time_full'] = row['local_date_time_full'][:13]
            # 求平均并更新其他列的值
            for col in df_mel_v2.columns:
                if col not in ['sort_order', 'local_date_time_full', 'name']:
                    # 如果是 str 类型，则将两个值放入一个列表中
                    if isinstance(row[col], str):
                        row[col] = [row[col], df_mel_v2.loc[idx + 1, col]]
                    # 如果是数值类型，则求平均
                    else:
                        row[col] = (row[col] + df_mel_v2.loc[idx + 1, col]) / 2
            # 将处理后的行添加到 rows_to_append 列表中
            rows_to_append.append(row)

    # 将 rows_to_append 列表中的行合并成 DataFrame，并添加到 processed_df 中
    processed_df = pd.concat([processed_df, pd.DataFrame(rows_to_append)], ignore_index=True)
    return processed_df
# 打印处理后的 DataFrame
processed_df = combine_1h(df_mel_v2)


In [24]:
processed_df.to_csv("data_sets/weather_static.csv")

In [25]:
processed_df

,name,local_date_time_full,lat,lon,apparent_t,delta_t,gust_kmh,gust_kt,air_temp,dewpt,press,press_qnh,press_msl,rain_trace,rel_hum,vis_km,wind_dir,wind_spd_kmh,wind_spd_kt
0,Melbourne CBD,2024-05-13-16,-37.8,145.0,16.85,2.75,8.5,4.5,16.70,11.70,1022.05,1022.05,1022.05,"[0.0, 0.0]",72.5,"[10, 10]","[CALM, S]",2.0,1.0
1,Melbourne CBD,2024-05-13-15,-37.8,145.0,16.15,3.15,12.0,6.5,17.15,11.30,1021.85,1021.85,1021.85,"[0.0, 0.0]",68.5,"[10, 10]","[SSW, SSW]",7.5,4.0
2,Melbourne CBD,2024-05-13-14,-37.8,145.0,17.10,3.30,9.0,5.0,17.70,11.65,1021.50,1021.50,1021.50,"[0.0, 0.0]",68.0,"[10, 10]","[S, SSE]",6.0,3.0
3,Melbourne CBD,2024-05-13-13,-37.8,145.0,13.40,1.70,19.5,10.5,14.95,11.80,1021.80,1021.80,1021.80,"[0.0, 0.0]",81.5,"[10, 10]","[SSW, SSW]",11.0,6.0
4,Melbourne CBD,2024-05-13-12,-37.8,145.0,15.30,3.10,18.5,10.0,16.70,10.95,1021.90,1021.90,1021.90,"[0.0, 0.0]",69.0,"[10, 10]","[SSW, SSW]",9.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Melbourne CBD,2024-05-10-21,-37.8,145.0,15.80,0.00,2.0,1.0,14.40,14.40,1028.50,1028.50,1028.50,"[0.6, 0.6]",100.0,"[10, 10]","[CALM, CALM]",0.0,0.0
77,Melbourne CBD,2024-05-10-20,-37.8,145.0,15.15,0.00,11.0,6.0,14.60,14.60,1028.40,1028.40,1028.40,"[0.6, 0.6]",100.0,"[10, 10]","[SE, S]",5.0,2.5
78,Melbourne CBD,2024-05-10-19,-37.8,145.0,16.20,0.00,6.5,3.5,14.80,14.80,1028.40,1028.40,1028.40,"[0.6, 0.6]",100.0,"[10, 10]","[S, CALM]",1.0,0.5
79,Melbourne CBD,2024-05-10-19,-37.8,145.0,16.10,0.00,5.0,2.5,14.75,14.75,1028.40,1028.40,1028.40,"[0.6, 0.6]",100.0,"[10, 5]","[CALM, S]",1.0,0.5


In [26]:
test = pd.read_csv("data_sets/weather_static.csv")
test

,Unnamed: 0,name,local_date_time_full,lat,lon,apparent_t,delta_t,gust_kmh,gust_kt,air_temp,dewpt,press,press_qnh,press_msl,rain_trace,rel_hum,vis_km,wind_dir,wind_spd_kmh,wind_spd_kt
0,0,Melbourne CBD,2024-05-13-16,-37.8,145.0,16.85,2.75,8.5,4.5,16.70,11.70,1022.05,1022.05,1022.05,"['0.0', '0.0']",72.5,"['10', '10']","['CALM', 'S']",2.0,1.0
1,1,Melbourne CBD,2024-05-13-15,-37.8,145.0,16.15,3.15,12.0,6.5,17.15,11.30,1021.85,1021.85,1021.85,"['0.0', '0.0']",68.5,"['10', '10']","['SSW', 'SSW']",7.5,4.0
2,2,Melbourne CBD,2024-05-13-14,-37.8,145.0,17.10,3.30,9.0,5.0,17.70,11.65,1021.50,1021.50,1021.50,"['0.0', '0.0']",68.0,"['10', '10']","['S', 'SSE']",6.0,3.0
3,3,Melbourne CBD,2024-05-13-13,-37.8,145.0,13.40,1.70,19.5,10.5,14.95,11.80,1021.80,1021.80,1021.80,"['0.0', '0.0']",81.5,"['10', '10']","['SSW', 'SSW']",11.0,6.0
4,4,Melbourne CBD,2024-05-13-12,-37.8,145.0,15.30,3.10,18.5,10.0,16.70,10.95,1021.90,1021.90,1021.90,"['0.0', '0.0']",69.0,"['10', '10']","['SSW', 'SSW']",9.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,76,Melbourne CBD,2024-05-10-21,-37.8,145.0,15.80,0.00,2.0,1.0,14.40,14.40,1028.50,1028.50,1028.50,"['0.6', '0.6']",100.0,"['10', '10']","['CALM', 'CALM']",0.0,0.0
77,77,Melbourne CBD,2024-05-10-20,-37.8,145.0,15.15,0.00,11.0,6.0,14.60,14.60,1028.40,1028.40,1028.40,"['0.6', '0.6']",100.0,"['10', '10']","['SE', 'S']",5.0,2.5
78,78,Melbourne CBD,2024-05-10-19,-37.8,145.0,16.20,0.00,6.5,3.5,14.80,14.80,1028.40,1028.40,1028.40,"['0.6', '0.6']",100.0,"['10', '10']","['S', 'CALM']",1.0,0.5
79,79,Melbourne CBD,2024-05-10-19,-37.8,145.0,16.10,0.00,5.0,2.5,14.75,14.75,1028.40,1028.40,1028.40,"['0.6', '0.6']",100.0,"['10', '5']","['CALM', 'S']",1.0,0.5
